In [21]:
import pandas as pd
import numpy as np
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import confusion_matrix


from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri


import matplotlib.pyplot as plt

import SMOTE
import feature_selector

from metrics import abcd
import metrices

import warnings
warnings.filterwarnings("ignore")

In [45]:
df = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/git_miner/data/hadoop_commit.pkl')
df1 = df[df['buggy'] == 1]
X1 = df1.commit_number
X2 = df1.parent
X = np.append(X1,X2)

In [46]:
df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/commit_guru/hadoop.csv')
df = df[df['commit_hash'].isin(X)]
df = df.drop(labels = ['commit_hash', 'author_name', 'author_date_unix_timestamp',
       'author_email', 'author_date', 'commit_message','classification', 'linked', 'contains_bug', 'fixes',
                      'fileschanged','glm_probability', 'rf_probability',
       'repository_id', 'issue_id', 'issue_date', 'issue_type'],axis=1)
df = df.dropna()
df = df[['ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age',
       'nuc', 'exp', 'rexp', 'sexp','fix']]
df = df.astype(np.float64)

In [47]:
df

,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,fix


In [20]:
df.to_csv('ant_updated.csv')

In [24]:
fs = feature_selector.featureSelector()
df,_selected,selected = fs.cfs_bfs(df)
cols = df.columns

KeyError: '[-1] not in index'

In [29]:
y = df.fix
y=y.astype('bool')
X = df.drop(labels=['fix'],axis=1)
train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=0.33, random_state=13)

ValueError: With n_samples=0, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [25]:
print(train_X.shape)
df_new = train_X
df_new['fix'] = train_y
smt = SMOTE.smote(train_X)
df = smt.run()
df.columns = cols
train_y = df.fix
train_y=train_y.astype('bool')
train_X = df.drop(labels=['fix'],axis=1)
print(train_X.shape)

(3992, 7)
(4778, 7)


In [26]:
clf = LogisticRegression(penalty= 'l2', multi_class= 'ovr',
                         C= 48.488, dual= True, fit_intercept= False, intercept_scaling =  54.623,
                         class_weight= None, solver= 'liblinear',
                         warm_start= False, max_iter= 1000)

In [ ]:
#clf = LogisticRegression(penalty='l1')
#clf = RandomForestClassifier(criterion = 'gini',n_estimators=100)
clf = SVC(kernel='linear', probability=True)
#clf = linear_model.SGDClassifier(loss = 'modified_huber',max_iter=1000, tol=1e-3,penalty='l1')
#clf = KNeighborsClassifier(n_neighbors=5,algorithm='kd_tree')
#kernel = 1.0 * RBF(1.0)
#clf = GaussianProcessClassifier(kernel=kernel,random_state=0)
clf.fit(train_X,train_y)
predicted = clf.predict(test_X)
#fpr, tpr, thresholds = metrics.roc_curve(test_y, predicted[:, 1], pos_label=1)
#print(metrics.auc(fpr, tpr))
print(classification_report(test_y, predicted))
#print(clf.coef_)
#plt.plot(fpr, tpr, lw=1, alpha=1)

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_y, predicted, labels=[0, 1]).ravel()
pd = int(100 * tp / (tp + fn + 1e-5))
pf = int(100 * fp / (fp + tn + 1e-5))
print(pd,pf)
beta = 5

In [8]:
meas = metrices.measures(test_y,predicted)

In [9]:
meas.get_g_score()

0.4167741935483871

In [ ]:
# For using with r package
train_df = train_X
test_df = test_X
train_df['fix'] = train_y.values
test_df['fix'] = test_y.values

with localconverter(ro.default_converter + pandas2ri.converter):
    train_df_r = ro.conversion.py2rpy(train_df)
    test_df_r = ro.conversion.py2rpy(test_df)

In [ ]:
stats = importr('stats', robject_translations={'format_perc': '_format_perc'})
base = importr('base')
sig_threshold = 0.05
def _isMetricSignificant(formula_metrics, metric,data):
    """
    Checks if adding a metric to the already significant metrics in formula_metrics in a GLM model is significant. If significant,
    and doesn't cause any previous metric in formula_metrics to become non significant, we return true. Otherwise, false.
    Note: The p-value is always given in the 4th column of the summary matrix!
    """
    sig_column = 4

    # Case 1: no existing metrics in the formula
    if len(formula_metrics) == 0:
        formula = "fix~" + metric
        fit = stats.glm(formula, data=data, family="binomial")
        summary = base.summary(fit)
      # Note - first row is the intercept information so we start at second row!

        try:
            metric_sig = summary.rx2('coefficients').rx(2,sig_column)[0] # Second row, 4th column of the summary matrix.
            if metric_sig <= sig_threshold:
                return True
            else:
                return False

        except:
            # If we have two metrics that are perfectly collinear it will not build the model with the metrics
            # and we will get an exception when trying to find the significance of *all values*. Indeed, do not add
            # this value to the model!
            return False

    # Case 2: existing metrics in the formula    
    else:
        num_metrics = len(formula_metrics)+2 # plus one for the new metric we are adding and one for intercept
        formula = "fix~" + "+".join(formula_metrics) + "+" + metric
        fit = stats.glm(formula, data=data, family="binomial")
        summary = base.summary(fit)

      # If any metric is now not significant, than we should not have added this metric to the formula
      # There are (intercept) + num_metrics rows in the matrix to check - starts at second row skipping intercept
        try: 
            for row in range(2,num_metrics+1):
                metric_sig = summary.rx2('coefficients').rx(row,sig_column)[0]
                if metric_sig > sig_threshold:
                    return False
            return True # old metrics added to model ARE significant still as well as the new one being tested

        except:
            # If we have two metrics that are perfectly collinear it will not build the model with the metrics
            # and we will get an exception when trying to find the significance of *all values*. Indeed, do not add
            # this value to the model!
            return False

        
def _getInterceptValue(coefs, data):
    """
    Return the Intercept value of a GLM model and the p-value
    Assumes that model can be built!
    """
    formula = "fix~" + "+".join(coefs)
    print(formula)
    fit = stats.glm(formula, data=data, family="binomial")
    summary = base.summary(fit)
    return summary.rx2('coefficients').rx(1)[0], summary.rx2('coefficients').rx(1,4)[0]

def _getCoefficients(formula_coefs,data):
    """
    Builds a GLM model with a formula based on the passed in coefficients and retuns a dictionary containing each
    coefficient with its value.
    """
    coef_dict = {} # a dict containing glm coefficients {name -> value}
    formula = "fix~" + "+".join(formula_coefs)
    fit = stats.glm(formula, data=data, family="binomial")

    for coef in formula_coefs:
        coef_dict[coef] = fit.rx2('coefficients').rx2(coef)[0]

    return coef_dict

In [ ]:
metrics_list = ["la","ld","lt","ns","nd","nf","ndev","age","nuc","exp","rexp","sexp","entropy"]
formula_metrics = []
for metric in metrics_list:
    if _isMetricSignificant(formula_metrics, metric,train_df_r):
        formula_metrics.append(metric)

In [ ]:
intercept_value, intercept_pvalue = _getInterceptValue(formula_metrics,train_df_r)
coefs = _getCoefficients(formula_metrics,train_df_r)

In [ ]:
predicted = []
risks = []
predicted_t = []
predicted_f = []
for commit in test_X.itertuples():
    coefs_sum = 0
    for coef_name in coefs:
        coefs_sum += (coefs[coef_name] * getattr(commit, coef_name))
    try:
        riskyness = 1/(1+ math.exp(-intercept_value-coefs_sum))
    except:
        riskyness = -1
    if riskyness >= 0.5:
        result = True
    else:
        result = False
    predicted.append(result)
    risks.append(riskyness)
y_train = test_y.values
for i in range(len(y_train)):
    if y_train[i]:
        predicted_t.append(risks[i])
    else:
        predicted_f.append(risks[i])

In [ ]:
print(classification_report(test_y, predicted))

In [ ]:
plt.hist(predicted_f)

In [ ]:
plt.hist(predicted_t)

In [ ]:
print(__doc__)

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

# #############################################################################
# Data IO and generation

# Import some data to play with
y = df.fix
y=y.astype('bool')
X = df.drop(labels=['fix'],axis=1)
X.reset_index(drop=True,inplace=True)
n_samples, n_features = X.shape


# #############################################################################
# Classification and ROC analysis

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10)
classifier = LogisticRegression(penalty='l1',class_weight = 'balanced')

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()